In [1547]:
import pandas as pd
import os

In [1548]:
import numpy as np
import plotly.express as px

In [1549]:
dfEx = pd.read_csv(os.getcwd()+'\\titanic\\'+'gender_submission_eg.csv')
dfTest = pd.read_csv(os.getcwd()+'\\titanic\\'+'test.csv')
dfTrain = pd.read_csv(os.getcwd()+'\\titanic\\'+'train.csv')

In [1550]:
dfEx

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [1551]:
dfTrain.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<h1>Tratamento de idades nulas</h1>

<h2>Train</h2>

In [1552]:
dfTrain['Age'].mean()

29.69911764705882

In [1553]:
dfTrain[dfTrain['Parch']==0]['Age'].mean()

32.17850287907869

In [1554]:
dfTrain[((dfTrain['Parch']>0) & ~((dfTrain['Name'].str.contains('Mrs')) | (dfTrain['Name'].str.contains('Mr'))))]['Age'].mean()

9.817065217391304

In [1555]:
dfTrain[(dfTrain['Parch']>0) & ((dfTrain['Name'].str.contains('Mrs')) | (dfTrain['Name'].str.contains('Mr')))]['Age'].mean()

35.01980198019802

In [1556]:
dfAge = dfTrain.copy()

In [1557]:
dfAge.loc[(dfAge['Parch']==0) & (dfAge['Age'].isnull()), 'Age'] = dfTrain['Age'].mean()

In [1558]:
dfAge.loc[(dfAge['Age'].isnull()) & (dfAge['Parch']>0) & ~((dfTrain['Name'].str.contains('Mrs')) | (dfTrain['Name'].str.contains('Mr'))),'Age'] = 9.817065217391304

In [1559]:
dfAge.loc[(dfAge['Age'].isnull()) & (dfAge['Parch']>0) & ((dfTrain['Name'].str.contains('Mrs')) | (dfTrain['Name'].str.contains('Mr'))),'Age'] = 35.01980198019802

In [1560]:
dfAge.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<h2>Test</h2>

In [1561]:
dfTest['Age'].mean()

30.272590361445783

In [1562]:
dfTest[dfTest['Parch']==0]['Age'].mean()

31.52834008097166

In [1563]:
dfTest[((dfTest['Parch']>0) & ~((dfTest['Name'].str.contains('Mrs')) | (dfTest['Name'].str.contains('Mr'))))]['Age'].mean()

12.05

In [1564]:
dfTest[(dfTest['Parch']>0) & ((dfTest['Name'].str.contains('Mrs')) | (dfTest['Name'].str.contains('Mr')))]['Age'].mean()

39.577777777777776

In [1565]:
dfTest.loc[(dfTest['Parch']==0) & (dfTest['Age'].isnull()), 'Age'] = dfTest['Age'].mean()

In [1566]:
dfTest.loc[(dfTest['Age'].isnull()) & (dfTest['Parch']>0) & ~((dfTest['Name'].str.contains('Mrs')) | (dfTest['Name'].str.contains('Mr'))),'Age'] = 12.05

In [1567]:
dfTest.loc[(dfTest['Age'].isnull()) & (dfTest['Parch']>0) & ((dfTest['Name'].str.contains('Mrs')) | (dfTest['Name'].str.contains('Mr'))),'Age'] = 39.577777777777776

In [1568]:
dfTest.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

<h1>Tratamento de cabines nulas</h1>

<h2>Train</h2>

In [1569]:
dfAgeCabin = dfAge.copy()

In [1570]:
#P1
A_q1 = dfAgeCabin[dfAgeCabin['Cabin'].str[:1] == 'A']['Fare'].quantile([0.25][0])
A_q3 = dfAgeCabin[dfAgeCabin['Cabin'].str[:1] == 'A']['Fare'].quantile([0.75][0])
B_mean = dfAgeCabin[dfAgeCabin['Cabin'].str[:1] == 'B']['Fare'].mean()

In [1571]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==1) & (dfAgeCabin['Fare'] >= A_q1)& (dfAgeCabin['Fare'] < A_q3),'Cabin']='A'

In [1572]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==1) & (dfAgeCabin['Fare'] >= A_q3) & (dfAgeCabin['Fare'] < B_mean),'Cabin']='B'

In [1573]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==1) & (dfAgeCabin['Fare'] >= B_mean),'Cabin']='C'

In [1574]:
dfAgeCabin.loc[(dfAgeCabin['Pclass']==1) & (dfAgeCabin['Cabin'].isnull()),'Cabin'] = 'E'

In [1575]:
#P2
D_mean = dfAgeCabin[dfAgeCabin['Cabin'].str[:1] == 'D']['Fare'].mean()
E_mean = dfAgeCabin[dfAgeCabin['Cabin'].str[:1] == 'E']['Fare'].mean()
F_mean = dfAgeCabin[dfAgeCabin['Cabin'].str[:1] == 'F']['Fare'].mean()
D_mean, E_mean, F_mean

(57.24457575757575, 38.64557083333333, 18.69679230769231)

In [1576]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==2) & (dfAgeCabin['Fare'] >= E_mean) & (dfAgeCabin['Fare'] < D_mean),'Cabin']='E'

In [1577]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==2) & (dfAgeCabin['Fare'] > D_mean),'Cabin']='D'

In [1578]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==2) & (dfAgeCabin['Fare'] >= F_mean) & (dfAgeCabin['Fare'] < E_mean),'Cabin']='F'

In [1579]:
dfAgeCabin.loc[(dfAgeCabin['Pclass']==2) & (dfAgeCabin['Cabin'].isnull()),'Cabin'] = 'E'

In [1580]:
#P3
G_max = dfAgeCabin[(dfAgeCabin['Pclass']==3) & (dfAgeCabin['Cabin'].str[:1]=='G')]['Fare'].max()
F_max = dfAgeCabin[(dfAgeCabin['Pclass']==3) & (dfAgeCabin['Cabin'].str[:1]=='F')]['Fare'].max()
F_max

22.3583

In [1581]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==3) & (dfAgeCabin['Fare'] <= G_max),'Cabin'] = 'G'

In [1582]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==3) & (dfAgeCabin['Fare'] > G_max) & (dfAgeCabin['Fare'] <= F_max),'Cabin'] ='F'

In [1583]:
dfAgeCabin.loc[(dfAgeCabin['Cabin'].isnull()) & (dfAgeCabin['Pclass']==3),'Cabin'] = 'E'

In [1584]:
dfAgeCabin.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

<h2>Test</h2>

In [1585]:
#P1
A_q1 = dfTest[dfTest['Cabin'].str[:1] == 'A']['Fare'].quantile([0.25][0])
A_q3 = dfTest[dfTest['Cabin'].str[:1] == 'A']['Fare'].quantile([0.75][0])
B_mean = dfTest[dfTest['Cabin'].str[:1] == 'B']['Fare'].mean()

In [1586]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==1) & (dfTest['Fare'] >= A_q1)& (dfTest['Fare'] < A_q3),'Cabin']='A'

In [1587]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==1) & (dfTest['Fare'] >= A_q3) & (dfTest['Fare'] < B_mean),'Cabin']='B'

In [1588]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==1) & (dfTest['Fare'] >= B_mean),'Cabin']='C'

In [1589]:
dfTest.loc[(dfTest['Pclass']==1) & (dfTest['Cabin'].isnull()),'Cabin'] = 'E'

In [1590]:
#P2
D_mean = dfTest[dfTest['Cabin'].str[:1] == 'D']['Fare'].mean()
E_mean = dfTest[dfTest['Cabin'].str[:1] == 'E']['Fare'].mean()
F_mean = dfTest[dfTest['Cabin'].str[:1] == 'F']['Fare'].mean()
D_mean, E_mean, F_mean

(42.251276923076915, 49.23294999999999, 17.076050000000002)

In [1591]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==2) & (dfTest['Fare'] >= E_mean) & (dfTest['Fare'] < D_mean),'Cabin']='E'

In [1592]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==2) & (dfTest['Fare'] > D_mean),'Cabin']='D'

In [1593]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==2) & (dfTest['Fare'] >= F_mean) & (dfTest['Fare'] < E_mean),'Cabin']='F'

In [1594]:
dfTest.loc[(dfTest['Pclass']==2) & (dfTest['Cabin'].isnull()),'Cabin'] = 'E'

In [1595]:
#P3
G_max = dfTest[(dfTest['Pclass']==3) & (dfTest['Cabin'].str[:1]=='G')]['Fare'].max()
F_max = dfTest[(dfTest['Pclass']==3) & (dfTest['Cabin'].str[:1]=='F')]['Fare'].max()
F_max

7.65

In [1596]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==3) & (dfTest['Fare'] <= G_max),'Cabin'] = 'G'

In [1597]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==3) & (dfTest['Fare'] > G_max) & (dfTest['Fare'] <= F_max),'Cabin'] ='F'

In [1598]:
dfTest.loc[(dfTest['Cabin'].isnull()) & (dfTest['Pclass']==3),'Cabin'] = 'E'

In [1599]:
dfTest.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin          0
Embarked       0
dtype: int64

In [1600]:
dfTest.loc[dfTest['Fare'].isnull(),'Fare'] = dfTest.loc[dfTest['Pclass']==3]['Fare'].mean()

In [1601]:
dfTest.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

<h1>Cabine</h1>

In [1602]:
dfAgeCabin['Cabin'] = dfAgeCabin['Cabin'].str[:1]

In [1603]:
list(dfAgeCabin['Cabin'])

['G',
 'C',
 'G',
 'C',
 'G',
 'G',
 'E',
 'F',
 'G',
 'F',
 'G',
 'C',
 'G',
 'E',
 'G',
 'E',
 'E',
 'E',
 'F',
 'G',
 'F',
 'D',
 'G',
 'A',
 'F',
 'E',
 'G',
 'C',
 'G',
 'G',
 'E',
 'B',
 'G',
 'E',
 'B',
 'B',
 'G',
 'G',
 'F',
 'G',
 'G',
 'F',
 'G',
 'E',
 'G',
 'G',
 'G',
 'G',
 'F',
 'F',
 'E',
 'G',
 'D',
 'F',
 'B',
 'C',
 'E',
 'G',
 'F',
 'E',
 'G',
 'B',
 'C',
 'E',
 'E',
 'G',
 'F',
 'G',
 'G',
 'G',
 'E',
 'E',
 'D',
 'G',
 'E',
 'F',
 'G',
 'G',
 'F',
 'G',
 'G',
 'G',
 'G',
 'A',
 'E',
 'G',
 'E',
 'G',
 'C',
 'G',
 'G',
 'G',
 'E',
 'F',
 'G',
 'G',
 'A',
 'D',
 'F',
 'F',
 'G',
 'G',
 'D',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'E',
 'C',
 'G',
 'G',
 'G',
 'G',
 'G',
 'G',
 'F',
 'B',
 'E',
 'D',
 'G',
 'F',
 'E',
 'D',
 'G',
 'G',
 'G',
 'F',
 'G',
 'G',
 'G',
 'G',
 'F',
 'E',
 'E',
 'D',
 'C',
 'G',
 'B',
 'G',
 'G',
 'G',
 'G',
 'E',
 'F',
 'G',
 'E',
 'F',
 'E',
 'E',
 'C',
 'G',
 'G',
 'G',
 'B',
 'G',
 'G',
 'G',
 'E',
 'G',
 'E',
 'G',
 'G',
 'E',
 'F',
 'E'

<h1>Tratamento de embarcados nulos</h1>

In [1604]:
dfAgeCabin.loc[dfAgeCabin['Embarked'].isnull(),'Embarked'] ='S'

In [1605]:
dfAgeCabin.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

<h1>Previsores x Classe</h1>

In [1606]:
dfAgeCabin.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,G,S


In [1607]:
X_train = dfAgeCabin.iloc[:,[2,4,5,6,7,9,10,11]].values

In [1608]:
y_train = dfAgeCabin.iloc[:,1].values

In [1609]:
X_train.shape, y_train.shape

((891, 8), (891,))

In [1610]:
X_test = dfTest.iloc[:,[1,3,4,5,6,8,9,10]].values

In [1611]:
X_train

array([[3, 'male', 22.0, ..., 7.25, 'G', 'S'],
       [1, 'female', 38.0, ..., 71.2833, 'C', 'C'],
       [3, 'female', 26.0, ..., 7.925, 'G', 'S'],
       ...,
       [3, 'female', 9.817065217391304, ..., 23.45, 'E', 'S'],
       [1, 'male', 26.0, ..., 30.0, 'C', 'C'],
       [3, 'male', 32.0, ..., 7.75, 'G', 'Q']], dtype=object)

<h1>Label Encoder</h1>

In [1612]:
X_train[0]

array([3, 'male', 22.0, 1, 0, 7.25, 'G', 'S'], dtype=object)

In [1613]:
from sklearn.preprocessing import LabelEncoder

In [1614]:
label_encoder_sex = LabelEncoder()
label_encoder_cabin = LabelEncoder()
label_encoder_embarked = LabelEncoder()

In [1615]:
X_train[:,1] = label_encoder_sex.fit_transform(X_train[:,1])
X_train[:,6] = label_encoder_cabin.fit_transform(X_train[:,6])
X_train[:,7] = label_encoder_embarked.fit_transform(X_train[:,7])
X_test[:,1] = label_encoder_sex.fit_transform(X_test[:,1])
X_test[:,6] = label_encoder_cabin.fit_transform(X_test[:,6])
X_test[:,7] = label_encoder_embarked.fit_transform(X_test[:,7])

In [1616]:
y_train

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [1617]:
type(X_train)

numpy.ndarray

In [1618]:
type(X_test)

numpy.ndarray

<h1>One Hot Encoder</h1>

In [1619]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [1620]:
onehotencoder_titanic = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0,1,3,4,6,7])], remainder = 'passthrough')

In [1621]:
onehotencoder_titanic.fit(X_test)

ColumnTransformer(remainder='passthrough',
                  transformers=[('OneHot', OneHotEncoder(),
                                 [0, 1, 3, 4, 6, 7])])

In [1622]:
X_train.shape, X_test.shape

((891, 8), (418, 8))

In [1623]:
X_test.shape

(418, 8)

In [1624]:
X_train = onehotencoder_titanic.transform(X_train).toarray()
X_test = onehotencoder_titanic.transform(X_test).toarray()

In [1625]:
X_train.shape, X_test.shape

((891, 106), (418, 106))

In [1626]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [1627]:
X_train.shape

(891, 106)

In [1628]:
X_test.shape

(418, 106)

In [1629]:
X_train

array([[-0.56568542, -0.51015154,  0.90258736, ...,  0.61583843,
        -0.56356674, -0.50244517],
       [ 1.76776695, -0.51015154, -1.10792599, ..., -1.62380254,
         0.64664267,  0.78684529],
       [-0.56568542, -0.51015154,  0.90258736, ...,  0.61583843,
        -0.26101439, -0.48885426],
       ...,
       [-0.56568542, -0.51015154,  0.90258736, ...,  0.61583843,
        -1.48506064, -0.17626324],
       [ 1.76776695, -0.51015154, -1.10792599, ..., -1.62380254,
        -0.26101439, -0.04438104],
       [-0.56568542, -0.51015154,  0.90258736, ..., -1.62380254,
         0.19281414, -0.49237783]])

In [1630]:
X_test

array([[-0.58655899, -0.5349335 ,  0.95782629, ..., -1.35067551,
         0.33558442, -0.49731093],
       [-0.58655899, -0.5349335 ,  0.95782629, ...,  0.74037028,
         1.31328455, -0.5121751 ],
       [-0.58655899,  1.86939125, -1.04403065, ..., -1.35067551,
         2.48652471, -0.46399919],
       ...,
       [-0.58655899, -0.5349335 ,  0.95782629, ...,  0.74037028,
         0.64844846, -0.50769362],
       [-0.58655899, -0.5349335 ,  0.95782629, ...,  0.74037028,
         0.0049333 , -0.49335289],
       [-0.58655899, -0.5349335 ,  0.95782629, ..., -1.35067551,
        -1.42036502, -0.23686347]])

<h1>KNN</h1>

In [1631]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier  = KNeighborsClassifier()

In [1632]:
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [1633]:
previsoes = knn_classifier.predict(X_test)

In [1634]:
previsoes

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,

In [1635]:
resultado = {'PassengerId':list(dfTest['PassengerId']), 'Survived':[previsao for previsao in previsoes]}
resultado = pd.DataFrame(resultado)
resultado.to_csv('resultado_knn.csv', index=False)

In [1636]:
resultado

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


<h1>Arvore de Decisão</h1>

In [1637]:
from sklearn.tree import DecisionTreeClassifier
arvore_titanic = DecisionTreeClassifier()

In [1638]:
arvore_titanic.fit(X_train,y_train)

DecisionTreeClassifier()

In [1639]:
previsoes = arvore_titanic.predict(X_test)

In [1640]:
previsoes

array([0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [1641]:
resultado = {'PassengerId':list(dfTest['PassengerId']), 'Survived':[previsao for previsao in previsoes]}
resultado = pd.DataFrame(resultado)
resultado.to_csv('resultado_tree.csv', index=False)

<h1>Random Forest</h1>

In [1642]:
from sklearn.ensemble import RandomForestClassifier

In [1643]:
random = RandomForestClassifier(n_estimators=75)

In [1644]:
random.fit(X_train, y_train)

RandomForestClassifier(n_estimators=75)

In [1645]:
previsoes = random.predict(X_test)

In [1646]:
previsoes

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [1647]:
resultado = {'PassengerId':list(dfTest['PassengerId']), 'Survived':[previsao for previsao in previsoes]}
resultado = pd.DataFrame(resultado)
resultado.to_csv('resultado_random.csv', index=False)

In [1648]:
resultado

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
